<a href="https://colab.research.google.com/github/aj9045j/sentiment_analysis/blob/main/tweet_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle
!pip install emoji
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 16.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from spellchecker import SpellChecker
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
import re
import emoji
import gensim
from gensim.utils import simple_preprocess
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
import pickle
import plotly.express as px

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 75% 61.0M/80.9M [00:00<00:00, 226MB/s]
100% 80.9M/80.9M [00:00<00:00, 209MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()

In [ ]:
df = pd.read_csv('training.1600000.processed.noemoticon.csv', encoding='latin1',names=['sentiment','id','date','qu','loc','tweet'])

In [ ]:
df = df.drop(['id', 'qu', 'loc','date'], axis=1)

In [ ]:
df.head()

,sentiment,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df['sentiment'].value_counts()

sentiment
0    800000
4    800000
Name: count, dtype: int64

preprocessing of datset


In [ ]:
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove @mentions
    text = re.sub(r'<[^>]+>', '', text)  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = text.lower()  # Convert to lowercase
    text = emoji.replace_emoji(text, replace='')  # Remove emojis
    return text

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    lemmatized_text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    return lemmatized_text


In [ ]:
top_250k = df.head(250000)
bottom_250k = df.tail(250000)
df = pd.concat([top_250k, bottom_250k])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sentiment  500000 non-null  int64 
 1   tweet      500000 non-null  object
dtypes: int64(1), object(1)
memory usage: 11.4+ MB


In [ ]:
df['sentiment'].value_counts()

sentiment
0    250000
4    250000
Name: count, dtype: int64

In [ ]:
tqdm.pandas()
df['tweet'] = df['tweet'].progress_apply(clean_text)

100%|██████████| 500000/500000 [01:17<00:00, 6476.22it/s] 


In [ ]:
spell = SpellChecker()

def correct_spell(text):
    corrected_words = []
    for word in text.split():
        corrected_word = spell.correction(word)
        if corrected_word is not None:
            corrected_words.append(corrected_word)
        else:
            corrected_words.append(word)  # Use the original word if correction is None
    corrected_text = ' '.join(corrected_words)
    return corrected_text


In [ ]:
df.iloc[18]

sentiment                                                    0
tweet         ahh ive always wanted to see rent love the so...
Name: 18, dtype: object

In [ ]:
df.head()

,sentiment,tweet
0,0,a thats a bummer you shoulda got david carr o...
1,0,is upset that he cant update his facebook by t...
2,0,i dived many times for the ball managed to sa...
3,0,my whole body feels itchy and like its on fire
4,0,no its not behaving at all im mad why am i he...


In [ ]:
tqdm.pandas()
df['tweet'] = df['tweet'].progress_apply(lemmatize)

100%|██████████| 500000/500000 [00:30<00:00, 16308.30it/s]


In [ ]:
df.head()

,sentiment,tweet
0,0,a thats a bummer you shoulda got david carr of...
1,0,is upset that he cant update his facebook by t...
2,0,i dived many time for the ball managed to save...
3,0,my whole body feel itchy and like it on fire
4,0,no it not behaving at all im mad why am i here...


In [ ]:
eng_stopworlds = stopwords.words('english')
def remove_stopwords(text):
  words = text.split()
  new_word = [word for word in words if word not in eng_stopworlds]
  new_text = ' '.join(new_word)
  return new_text

In [ ]:
tqdm.pandas()
df['tweet'] = df['tweet'].progress_apply(remove_stopwords)

100%|██████████| 500000/500000 [00:17<00:00, 28903.97it/s]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, 0 to 1599999
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   sentiment  500000 non-null  int64 
 1   tweet      500000 non-null  object
dtypes: int64(1), object(1)
memory usage: 11.4+ MB


In [ ]:
total = []
for x in df['tweet']:
    raw = sent_tokenize(x)
    for r in raw:
        total.append(simple_preprocess(r))

In [ ]:
total

[['thats', 'bummer', 'shoulda', 'got', 'david', 'carr', 'third', 'day'],
 ['upset',
  'cant',
  'update',
  'facebook',
  'texting',
  'might',
  'cry',
  'result',
  'school',
  'today',
  'also',
  'blah'],
 ['dived', 'many', 'time', 'ball', 'managed', 'save', 'rest', 'go', 'bound'],
 ['whole', 'body', 'feel', 'itchy', 'like', 'fire'],
 ['behaving', 'im', 'mad', 'cant', 'see'],
 ['whole', 'crew'],
 ['need', 'hug'],
 ['hey',
  'long',
  'time',
  'see',
  'yes',
  'rain',
  'bit',
  'bit',
  'lol',
  'im',
  'fine',
  'thanks',
  'hows'],
 ['nope', 'didnt'],
 ['que', 'muera'],
 ['spring', 'break', 'plain', 'city', 'snowing'],
 ['repierced', 'ear'],
 ['couldnt', 'bear', 'watch', 'thought', 'ua', 'loss', 'wa', 'embarrassing'],
 ['count', 'idk', 'either', 'never', 'talk', 'anymore'],
 ['wouldve',
  'first',
  'didnt',
  'gun',
  'really',
  'though',
  'zac',
  'snyders',
  'doucheclown'],
 ['wish', 'got', 'watch', 'miss', 'wa', 'premiere'],
 ['hollis',
  'death',
  'scene',
  'hurt',
  

In [ ]:
model = gensim.models.Word2Vec(
    vector_size=300,
    window=10,
    min_count=2,
)

In [ ]:
model.build_vocab(total)

In [ ]:
model.train(total,total_examples=model.corpus_count,epochs=10)

(31607901, 35573140)

In [ ]:
model.wv.most_similar('good')

[('great', 0.5365612506866455),
 ('rough', 0.48381075263023376),
 ('bad', 0.4734031856060028),
 ('goood', 0.4711209237575531),
 ('excellent', 0.45868051052093506),
 ('gooood', 0.443031907081604),
 ('fantastic', 0.4321915805339813),
 ('terrible', 0.4273611009120941),
 ('interesting', 0.4161373972892761),
 ('shitty', 0.41411933302879333)]

In [ ]:
model.wv.most_similar('best')

[('greatest', 0.6339297890663147),
 ('bestest', 0.5740532279014587),
 ('shittest', 0.5640168786048889),
 ('dumbest', 0.5410204529762268),
 ('worst', 0.5336258411407471),
 ('coolest', 0.512775719165802),
 ('hottest', 0.5004738569259644),
 ('funniest', 0.50033038854599),
 ('craziest', 0.4942151606082916),
 ('saddest', 0.49398624897003174)]

In [ ]:
pca = PCA(n_components=3)

In [ ]:
x = pca.fit_transform(model.wv.get_normed_vectors())

In [ ]:
y = model.wv.index_to_key

In [ ]:

fig = px.scatter_3d(x[:200],x=0,y=1,z=2,color=y[:200])
fig.show()

In [ ]:

filename = "tweet_pred_word2vec1.sav"
pickle.dump(model,open(filename,'wb'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import drive
import shutil
shutil.copy2('tweet_pred_word2vec1.sav', '/content/drive/My Drive/')

Mounted at /content/drive


'/content/drive/My Drive/tweet_pred_word2vec1.sav'

now make classification using random forest

In [ ]:
empty = []
def document_con(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    if not doc:

        # If no words are found in the model's vocabulary, return a vector of zeros
        return np.zeros(model.vector_size)

    return np.mean(model.wv[doc],axis=0)

In [ ]:
x_val = []
for xy in tqdm(df['tweet']):
    x_val.append(document_con(xy))

100%|██████████| 500000/500000 [21:15<00:00, 392.02it/s]


In [ ]:
path = '/content/drive/My Drive/x_val1.pkl'

# Save x_val to a file in Google Drive
with open(path, 'wb') as f:
    pickle.dump(x_val, f)

# To load it back later
# with open(path, 'rb') as f:
#     x_val_loaded = pickle.load(f)


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x_val,df['sentiment'],test_size=0.2,random_state=42)

In [ ]:
y_train.value_counts()

sentiment
0    200004
4    199996
Name: count, dtype: int64

In [ ]:
rf = RandomForestRegressor()
rf.fit(x_train,y_train)



In [ ]:
y_pred = rf.predict(x_test)

In [ ]:
def conv_val(num):
    if(num >= 2):
        return 4
    else:
        return 0

In [ ]:
y_pred_binary = [conv_val(num) for num in y_pred]
y_test.value_counts()

In [ ]:
accuracy_score(y_pred_binary,y_test)

In [ ]:
filename = "tweet_pred_senti1.sav"
pickle.dump(rf,open(filename,'wb'))

In [ ]:
shutil.copy2('tweet_pred_senti.sav1', '/content/drive/My Drive/')

In [ ]:
i = 0
while (true):
  i++


In [ ]:
print("hey completed ")

while(true):
y